(Introduce the project)
(Introduce the dataset)

## Gather

(import statements)

In [ ]:
import json
import numpy as np
import pandas as pd
import requests
import tweepy
from timeit import default_timer as timer
from tweepy import OAuthHandler

In [ ]:
# Load twitter archive file
archive = pd.read_csv('twitter-archive-enhanced.csv')
archive.head()

(load image_predictions.tsv)

In [ ]:
# Download image-predictions file
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
with open('image_predictions.tsv', mode='wb') as file:
    file.write(response.content)

In [ ]:
# Import image predictions file
image_predictions = pd.read_csv('image_predictions.tsv', sep='\t')
image_predictions.head()

(use twitter api for json data)
(test on small sample, once working get complete data set once and then comment out)
(temp: load from a saved local file)

In [ ]:
# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
consumer_key = 'g2ND7axSfMh4x6NastwdByJQd'
consumer_secret = 'V14qYWCX7aB3nnjoigO1B6hle1D9MOultmA8qc8g2IiQmHQByD'
access_token = '736183273-d5LSGYOp5lxQNPGLN3fTUsEgoNCYElxHIcyVSVHd'
access_secret = 'Q1gRx9PmmufVjl2QyMsKv3sUXJ5RLUn7GaSpugJRv7QXP'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
# Tweet IDs for which to gather additional data via Twitter's API
tweet_ids = archive.tweet_id.values
len(tweet_ids)

In [ ]:
# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
# NOTE: Uncomment out for submission
'''
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)
'''

In [ ]:
# Temp - save fails_dict to file for working with
# NOTE: Delete prior to submission
'''
import csv
fails_list = []
for key in fails_dict.keys():
    fails_list.append(key)
fails_list
with open('fails_dict.csv', 'w') as outfile:
    wr = csv.writer(outfile, dialect='excel')
    wr.writerow(fails_list)
'''

In [ ]:
# Working - uncomment out if necessary
# NOTE: Delete prior to submission
'''
failures = []
with open('fails_dict.csv', 'r') as f:
    reader = csv.reader(f)
    for line in reader:
        failures.append(line)
'''

In [ ]:
# Load extended tweet data
tweets = []
with open('tweet_json.txt') as json_file:
    for line in json_file:
        tweets.append(json.loads(line))
extended_data = pd.DataFrame(tweets)
extended_data.head()

## Assess

(Explore each using .info)

#### Quality
##### `twitter_archive_enhanced` table
- x 
- x
- x

##### `image_predictions` table
- x
- x

##### `additional_data` table
- x
- x
- x

#### Tidiness
- x
- x

## Clean

(make copies of datasources)

### (Problem 1)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 2)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 3)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 4)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 5)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 6)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 7)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 8)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 9)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 10)

(Problem: ...)

##### Define
x

##### Code

##### Test

## Storage

(store each table as a csv file with main file called tiwtter_archive_master.csv)

## Analysis

(make at least 3 insights)